In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from baseline import *
import numpy as np
import torch.optim as optim

In [22]:
# model
model = CNNBaseline()

# other external model settings
num_epochs = 10
batch_size = 8
train_acc, train_loss, val_acc, val_loss = [], [], [], []

In [23]:

# data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageFolder('./data', transform=transform)

train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [24]:
# setting metrics
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [25]:
# training and validating the model
for epoch in range(num_epochs):
    num_correct = 0
    epoch_train_loss = 0
    model.train()
    num_batches = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        _, predictions = outputs.max(1)
        num_correct += (predictions == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_train_loss += loss.item()
        
        num_batches = batch_idx
    
    train_acc.append(num_correct / num_batches)
    train_loss.append(epoch_train_loss / num_batches)
    
    
    model.eval()
    epoch_val_loss = 0
    epoch_val_correct = 0
    num_val_batches = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predictions = outputs.max(1)
            epoch_val_correct += (predictions == labels).sum().item()
            loss = criterion(outputs, labels)
            epoch_val_loss += loss.item()
            
            num_val_batches += 1

    epoch_val_acc = epoch_val_correct / len(val_loader.dataset)
    epoch_val_loss /= num_val_batches
    
    val_loss.append(epoch_val_loss)  
    val_acc.append(epoch_val_acc)    
    
    
        
    print('Epoch [{}/{}], Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch+1, num_epochs, train_loss[-1], val_loss[-1], val_acc[-1]))


Epoch [1/10], Loss: 3.4710, Val Loss: 1.5368, Val Acc: 0.2750
Epoch [2/10], Loss: 0.9655, Val Loss: 1.7911, Val Acc: 0.4750
Epoch [3/10], Loss: 0.3426, Val Loss: 0.9494, Val Acc: 0.7000
Epoch [4/10], Loss: 0.1585, Val Loss: 0.6296, Val Acc: 0.7500
Epoch [5/10], Loss: 0.0471, Val Loss: 0.4436, Val Acc: 0.7750
Epoch [6/10], Loss: 0.0037, Val Loss: 0.4698, Val Acc: 0.8000
Epoch [7/10], Loss: 0.0011, Val Loss: 0.4513, Val Acc: 0.8250
Epoch [8/10], Loss: 0.0008, Val Loss: 0.4287, Val Acc: 0.8250
Epoch [9/10], Loss: 0.0007, Val Loss: 0.4108, Val Acc: 0.8250
Epoch [10/10], Loss: 0.0005, Val Loss: 0.4216, Val Acc: 0.8000


In [28]:
# Save model
torch.save(model.state_dict(), './models/final_model.h5')

In [32]:
# Load model
device = torch.device("cuda")
model = CNNBaseline()
model.load_state_dict(torch.load('./models/final_model.h5'))
# model = model.to(device)

<All keys matched successfully>

In [33]:
# Testing model
test_correct = 0
test_loss = 0

with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predictions = outputs.max(1)
            test_correct += (predictions == labels).sum().item()
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            num_val_batches += 1

        test_acc = test_correct / len(test_loader.dataset)
    
        
        print('Test Loss: {:.4f}, Test Acc: {:.4f}'.format(test_loss, test_acc))

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx